# Clinical Trials Gen AI Workshop Part 1


**Note:** This notebook was designed to be run in a JupyterLab space in SageMaker Studio running *Sagemaker Distribution 1.4* on an *ml.t3.medium* instance with 5GB of storage, although other configurations may be supported.

## Introduction

This workshop is designed to show you how to use Amazon Bedrock and Large Language Models (LLM) to solve a real-world problem described in the user persona below.

### User persona

AnyCompany is a healthcare startup that provides insights based on patient history. They offer a solution that can be embedded into an EMR, so their customers include EMR vendors and healthcare providers. They offer their solution as a SaaS model, and their infrastructure is cloud native and built leveraging serverless to provide elasticity and scalability. The product team recently flagged the need for the company to offer **Cohort Identification** as a service, and their first investment is to build a **Clinical Trials** matching feature. The business is attempting to evaluate if a clinical trial is suitable for a patient given the patient's longitudinal records.

As a data scientist recently hired by AnyCompany, you got pulled into a tiger team that has the goal of (i) validating if this project is feasible and (ii) building a working proof of concept.

If your assignment succeeds, this feature will be added to AnyCompany's next big release, which is coming out soon.

The clock is ticking, so let's start building!


## Setup

First, import some pre-requisite libraries

In [ ]:
import requests
from xml.etree import ElementTree
import boto3
import json
print("All pre-requisite libraries successfully installed")

Setting VERBOSE_LOGGING to ***True*** will generate a lot more cell output but may make it easier to visualize full prompts and can be a useful troubleshooting tool. We use ***False*** by default to make the output easier to read

In [ ]:
VERBOSE_LOGGING = False
print(f"Current verbose logging setting is {VERBOSE_LOGGING}")

## 1. Clinical Trial Information Processing

```mermaid
flowchart LR
    A[Clinical Trial XML] -->|Parse| B[Trial Criteria JSON]

    style A fill:#f9f,stroke:#333
    style B fill:#bbf,stroke:#333
```

### 1.1 Helper function to convert XML to JSON

We define a helper function below called parseXmlToJson.

It converts an XML like


    <eligibility>
        <criteria>
            <textblock>
                TEXT ABOUT ELIGIBILITY CRITERIA
            </textblock>
        </criteria>
    </eligibility>


to

    {
    	"eligibility": {
    		"criteria": {
    			"textblock": "TEXT ABOUT ELIGIBILITY CRITERIA"
    		}
    	}
    }

In [ ]:
def parseXmlToJson(xml):
  response = {}

  for child in list(xml):
    if len(list(child)) > 0:
      response[child.tag] = parseXmlToJson(child)
    else:
      response[child.tag] = child.text or ''

  return response

### 1.2 Download the clinical trial information

We identify a clinical trial of interest. 

Here we've chosen [NCT03095521](https://classic.clinicaltrials.gov/ct2/show/NCT03095521) titled ***A Local Clinical Study for the Comparative Evaluation of Efficacy and Safety of Angal, Lozenges [Menthol] and ANTI-ANGIN® FORMULA, Lozenges, in Treatment of Patients With a Sore Throat***

In [ ]:
# store the clinicaltrials.gov id in the variable
clinical_trial_id = 'NCT03095521'

# We download the clinical trial information
# from clinicaltrials.gov in xml format.
query_string = f"https://clinicaltrials.gov/ct2/show/{clinical_trial_id}?displayxml=true"
response = requests.get(query_string)

# Parse the XML to JSON object using the helper function
# we defined above
events = ElementTree.fromstring(response.content)
jsonObj = parseXmlToJson(events)

# The clinical trials website/information download includes 
# a lot of information about the clinical trial, we are only 
# interested in the criteria for the study

criteria = jsonObj["eligibility"]["criteria"]["textblock"]
print(criteria)

## 2. Download Patient Longitudinal Records

```mermaid
flowchart LR
    C[Patient Resource] -->|Parse| D[Patient Profile]
    E[Patient Encounters] -->|Parse| D
    F[Patient Observations] -->|Parse| D
    G[Patient Conditions] -->|Parse| D
```

Here we defined some sample patient information. This was generated using [Synthea](https://synthetichealth.github.io/synthea/).

This section could be replaced by making a query to a FHIR datastore such as [AWS Healthlake](https://aws.amazon.com/healthlake/)

In [ ]:
sample_patient_resource = """{"identifier":[{"system":"https://github.com/synthetichealth/synthea","value":"04c704c4-5d2d-4308-9c33-1690a6e47a6b"},{"system":"http://hospital.smarthealthit.org","type":{"coding":[{"system":"http://terminology.hl7.org/CodeSystem/v2-0203","code":"MR","display":"Medical Record Number"}],"text":"Medical Record Number"},"value":"04c704c4-5d2d-4308-9c33-1690a6e47a6b"},{"system":"http://hl7.org/fhir/sid/us-ssn","type":{"coding":[{"system":"http://terminology.hl7.org/CodeSystem/v2-0203","code":"SS","display":"Social Security Number"}],"text":"Social Security Number"},"value":"999-83-3466"},{"system":"urn:oid:2.16.840.1.113883.4.3.25","type":{"coding":[{"system":"http://terminology.hl7.org/CodeSystem/v2-0203","code":"DL","display":"Driver's License"}],"text":"Driver's License"},"value":"S99998319"},{"system":"http://standardhealthrecord.org/fhir/StructureDefinition/passportNumber","type":{"coding":[{"system":"http://terminology.hl7.org/CodeSystem/v2-0203","code":"PPN","display":"Passport Number"}],"text":"Passport Number"},"value":"X75739588X"}],"extension":[{"valueString":"Jacquelin132 Zemlak964","url":"http://hl7.org/fhir/StructureDefinition/patient-mothersMaidenName"},{"valueAddress":{"country":"US","city":"Haverhill","state":"Massachusetts"},"url":"http://hl7.org/fhir/StructureDefinition/patient-birthPlace"},{"valueDecimal":0.00395599232589674,"url":"http://synthetichealth.github.io/synthea/disability-adjusted-life-years"},{"valueDecimal":21.996044007674104,"url":"http://synthetichealth.github.io/synthea/quality-adjusted-life-years"}],"address":[{"country":"US","extension":[{"extension":[{"valueDecimal":42.153621929974896,"url":"latitude"},{"valueDecimal":-71.33316034414361,"url":"longitude"}],"url":"http://hl7.org/fhir/StructureDefinition/geolocation"}],"city":"Millis","line":["738 Cassin Club"],"state":"Massachusetts"}],"gender":"male","multipleBirthBoolean":false,"birthDate":"1997-11-22","name":[{"given":["Dexter530"],"use":"official","prefix":["Mr."],"family":"Little434"}],"telecom":[{"system":"phone","use":"home","value":"555-132-1111"}],"id":"04c704c4-5d2d-4308-9c33-1690a6e47a6b","text":{"div":"<div xmlns=\"http://www.w3.org/1999/xhtml\">Generated by <a href=\"https://github.com/synthetichealth/synthea\">Synthea</a>.Version identifier: master-branch-latest\n .   Person seed: -1028339658756901078  Population seed: 1593457889422</div>","status":"generated"},"communication":[{"language":{"coding":[{"system":"urn:ietf:bcp:47","code":"en-US","display":"English"}],"text":"English"}}],"maritalStatus":{"coding":[{"system":"http://terminology.hl7.org/CodeSystem/v3-MaritalStatus","code":"S","display":"Never Married"}],"text":"Never Married"},"resourceType":"Patient","meta":{"lastUpdated":"2023-11-17T01:57:03.687Z"}}"""
sample_patient_encounters = """{"resourceType":"Bundle","type":"searchset","entry":[{"resource":{"resourceType":"Encounter","id":"0fce7a0a-3035-40c8-a129-50487fc75409","meta":{"lastUpdated":"2023-11-17T01:57:03.687Z"},"status":"finished","class":{"system":"http://terminology.hl7.org/CodeSystem/v3-ActCode","code":"AMB"},"type":[{"coding":[{"system":"http://snomed.info/sct","code":"162673000","display":"General examination of patient (procedure)"}],"text":"General examination of patient (procedure)"}],"subject":{"reference":"Patient/04c704c4-5d2d-4308-9c33-1690a6e47a6b","display":"Mr. Dexter530 Little434"},"participant":[{"type":[{"coding":[{"system":"http://terminology.hl7.org/CodeSystem/v3-ParticipationType","code":"PPRF","display":"primary performer"}],"text":"primary performer"}],"period":{"start":"2020-01-25T02:37:32-08:00","end":"2020-01-25T02:52:32-08:00"},"individual":{"reference":"Practitioner/57b4d535-d416-37e9-9419-ca1790d88145","display":"Dr. Lovie151 Jerde200"}}],"period":{"start":"2020-01-25T02:37:32-08:00","end":"2020-01-25T02:52:32-08:00"},"serviceProvider":{"reference":"Organization/59029440-c787-39fb-8ad9-8e2e14ae018a","display":"PCP180419"}},"search":{"mode":"match"}},{"resource":{"resourceType":"Encounter","id":"2349e4b5-5163-425d-a444-b0e459a0260d","meta":{"lastUpdated":"2023-11-17T01:57:03.687Z"},"status":"finished","class":{"system":"http://terminology.hl7.org/CodeSystem/v3-ActCode","code":"AMB"},"type":[{"coding":[{"system":"http://snomed.info/sct","code":"185345009","display":"Encounter for symptom (procedure)"}],"text":"Encounter for symptom (procedure)"}],"subject":{"reference":"Patient/04c704c4-5d2d-4308-9c33-1690a6e47a6b","display":"Mr. Dexter530 Little434"},"participant":[{"type":[{"coding":[{"system":"http://terminology.hl7.org/CodeSystem/v3-ParticipationType","code":"PPRF","display":"primary performer"}],"text":"primary performer"}],"period":{"start":"2020-03-10T03:37:32-07:00","end":"2020-03-10T05:14:32-07:00"},"individual":{"reference":"Practitioner/b1027edf-0f61-3cc7-bad2-8922a5b4b90b","display":"Dr. Latrisha74 Bahringer146"}}],"period":{"start":"2020-03-10T03:37:32-07:00","end":"2020-03-10T05:14:32-07:00"},"serviceProvider":{"reference":"Organization/3bd5eda0-16da-3ba5-8500-4dfd6ae118b8","display":"NORWOOD HOSPITAL"}},"search":{"mode":"match"}},{"resource":{"resourceType":"Encounter","id":"cb5a9c67-6e54-49de-b5ad-c733cf348913","meta":{"lastUpdated":"2023-11-17T01:57:03.689Z"},"status":"finished","class":{"system":"http://terminology.hl7.org/CodeSystem/v3-ActCode","code":"AMB"},"type":[{"coding":[{"system":"http://snomed.info/sct","code":"185345009","display":"Encounter for symptom"}],"text":"Encounter for symptom"}],"subject":{"reference":"Patient/04c704c4-5d2d-4308-9c33-1690a6e47a6b","display":"Mr. Dexter530 Little434"},"participant":[{"type":[{"coding":[{"system":"http://terminology.hl7.org/CodeSystem/v3-ParticipationType","code":"PPRF","display":"primary performer"}],"text":"primary performer"}],"period":{"start":"2018-02-19T02:37:32-08:00","end":"2018-02-19T02:52:32-08:00"},"individual":{"reference":"Practitioner/b1027edf-0f61-3cc7-bad2-8922a5b4b90b","display":"Dr. Latrisha74 Bahringer146"}}],"period":{"start":"2018-02-19T02:37:32-08:00","end":"2018-02-19T02:52:32-08:00"},"reasonCode":[{"coding":[{"system":"http://snomed.info/sct","code":"444814009","display":"Viral sinusitis (disorder)"}]}],"serviceProvider":{"reference":"Organization/3bd5eda0-16da-3ba5-8500-4dfd6ae118b8","display":"NORWOOD HOSPITAL"}},"search":{"mode":"match"}},{"resource":{"resourceType":"Encounter","id":"dc3fa5ac-efdb-43e0-adad-b38abfc94458","meta":{"lastUpdated":"2023-11-17T01:57:03.689Z"},"status":"finished","class":{"system":"http://terminology.hl7.org/CodeSystem/v3-ActCode","code":"AMB"},"type":[{"coding":[{"system":"http://snomed.info/sct","code":"185345009","display":"Encounter for symptom"}],"text":"Encounter for symptom"}],"subject":{"reference":"Patient/04c704c4-5d2d-4308-9c33-1690a6e47a6b","display":"Mr. Dexter530 Little434"},"participant":[{"type":[{"coding":[{"system":"http://terminology.hl7.org/CodeSystem/v3-ParticipationType","code":"PPRF","display":"primary performer"}],"text":"primary performer"}],"period":{"start":"2018-10-06T03:37:32-07:00","end":"2018-10-06T03:52:32-07:00"},"individual":{"reference":"Practitioner/b1027edf-0f61-3cc7-bad2-8922a5b4b90b","display":"Dr. Latrisha74 Bahringer146"}}],"period":{"start":"2018-10-06T03:37:32-07:00","end":"2018-10-06T03:52:32-07:00"},"reasonCode":[{"coding":[{"system":"http://snomed.info/sct","code":"43878008","display":"Streptococcal sore throat (disorder)"}]}],"serviceProvider":{"reference":"Organization/3bd5eda0-16da-3ba5-8500-4dfd6ae118b8","display":"NORWOOD HOSPITAL"}},"search":{"mode":"match"}}]}"""
sample_patient_observations = """
"resourcetype","id","meta","implicitrules","_implicitrules","language","_language","text","extension","modifierextension","identifier","basedon","partof","status","_status","category","code","subject","focus","encounter","effectivedatetime","_effectivedatetime","effectiveperiod","effectivetiming","effectiveinstant","_effectiveinstant","issued","_issued","performer","valuequantity","valuecodeableconcept","valuestring","_valuestring","valueboolean","_valueboolean","valueinteger","_valueinteger","valuerange","valueratio","valuesampleddata","valuetime","_valuetime","valuedatetime","_valuedatetime","valueperiod","dataabsentreason","interpretation","note","bodysite","method","specimen","device","referencerange","hasmember","derivedfrom","component"
"Observation","76719a85-960d-4fd3-bf71-355a73ed4ade","{id=null, extension=null, versionId=null, _versionId=null, lastUpdated=2023-11-17T01:57:03.688Z, _lastUpdated=null, source=null, _source=null, profile=null, security=null, tag=null}",,,,,,,,,,,"final",,"[{id=null, extension=null, coding=[{id=null, extension=null, system=http://terminology.hl7.org/CodeSystem/observation-category, _system=null, version=null, _version=null, code=laboratory, _code=null, display=laboratory, _display=null, userSelected=null, _userSelected=null}], text=null, _text=null}]","{id=null, extension=null, coding=[{id=null, extension=null, system=http://loinc.org, _system=null, version=null, _version=null, code=785-6, _code=null, display=MCH [Entitic mass] by Automated count, _display=null, userSelected=null, _userSelected=null}], text=MCH [Entitic mass] by Automated count, _text=null}","{id=null, extension=null, reference=Patient/04c704c4-5d2d-4308-9c33-1690a6e47a6b, _reference=null, type=null, _type=null, identifier=null, display=null, _display=null}",,"{id=null, extension=null, reference=Encounter/0fce7a0a-3035-40c8-a129-50487fc75409, _reference=null, type=null, _type=null, identifier=null, display=null, _display=null}","2020-01-25T02:37:32-08:00",,,,,,"2020-01-25T02:37:32.457-08:00",,,"{id=null, extension=null, value=29, _value=null, comparator=null, _comparator=null, unit=pg, _unit=null, system=http://unitsofmeasure.org, _system=null, code=pg, _code=null}",,,,,,,,,,,,,,,,,,,,,,,,,,
"Observation","5b51922d-9689-4ee2-8412-93dde2a2d281","{id=null, extension=null, versionId=null, _versionId=null, lastUpdated=2023-11-17T01:57:03.688Z, _lastUpdated=null, source=null, _source=null, profile=null, security=null, tag=null}",,,,,,,,,,,"final",,"[{id=null, extension=null, coding=[{id=null, extension=null, system=http://terminology.hl7.org/CodeSystem/observation-category, _system=null, version=null, _version=null, code=vital-signs, _code=null, display=vital-signs, _display=null, userSelected=null, _userSelected=null}], text=null, _text=null}]","{id=null, extension=null, coding=[{id=null, extension=null, system=http://loinc.org, _system=null, version=null, _version=null, code=8867-4, _code=null, display=Heart rate, _display=null, userSelected=null, _userSelected=null}], text=Heart rate, _text=null}","{id=null, extension=null, reference=Patient/04c704c4-5d2d-4308-9c33-1690a6e47a6b, _reference=null, type=null, _type=null, identifier=null, display=null, _display=null}",,"{id=null, extension=null, reference=Encounter/2349e4b5-5163-425d-a444-b0e459a0260d, _reference=null, type=null, _type=null, identifier=null, display=null, _display=null}","2020-03-10T03:37:32-07:00",,,,,,"2020-03-10T03:37:32.457-07:00",,,"{id=null, extension=null, value=168, _value=null, comparator=null, _comparator=null, unit=/min, _unit=null, system=http://unitsofmeasure.org, _system=null, code=/min, _code=null}",,,,,,,,,,,,,,,,,,,,,,,,,,
"Observation","4590794d-71fb-4019-b322-45773b73fa34","{id=null, extension=null, versionId=null, _versionId=null, lastUpdated=2023-11-17T01:57:03.687Z, _lastUpdated=null, source=null, _source=null, profile=null, security=null, tag=null}",,,,,,,,,,,"final",,"[{id=null, extension=null, coding=[{id=null, extension=null, system=http://terminology.hl7.org/CodeSystem/observation-category, _system=null, version=null, _version=null, code=vital-signs, _code=null, display=vital-signs, _display=null, userSelected=null, _userSelected=null}], text=null, _text=null}]","{id=null, extension=null, coding=[{id=null, extension=null, system=http://loinc.org, _system=null, version=null, _version=null, code=72514-3, _code=null, display=Pain severity - 0-10 verbal numeric rating [Score] - Reported, _display=null, userSelected=null, _userSelected=null}], text=Pain severity - 0-10 verbal numeric rating [Score] - Reported, _text=null}","{id=null, extension=null, reference=Patient/04c704c4-5d2d-4308-9c33-1690a6e47a6b, _reference=null, type=null, _type=null, identifier=null, display=null, _display=null}",,"{id=null, extension=null, reference=Encounter/0fce7a0a-3035-40c8-a129-50487fc75409, _reference=null, type=null, _type=null, identifier=null, display=null, _display=null}","2020-01-25T02:37:32-08:00",,,,,,"2020-01-25T02:37:32.457-08:00",,,"{id=null, extension=null, value=4, _value=null, comparator=null, _comparator=null, unit={score}, _unit=null, system=http://unitsofmeasure.org, _system=null, code={score}, _code=null}",,,,,,,,,,,,,,,,,,,,,,,,,,
"Observation","9c84eeed-2263-47fd-9580-b7bde281f622","{id=null, extension=null, versionId=null, _versionId=null, lastUpdated=2023-11-17T01:57:03.688Z, _lastUpdated=null, source=null, _source=null, profile=null, security=null, tag=null}",,,,,,,,,,,"final",,"[{id=null, extension=null, coding=[{id=null, extension=null, system=http://terminology.hl7.org/CodeSystem/observation-category, _system=null, version=null, _version=null, code=laboratory, _code=null, display=laboratory, _display=null, userSelected=null, _userSelected=null}], text=null, _text=null}]","{id=null, extension=null, coding=[{id=null, extension=null, system=http://loinc.org, _system=null, version=null, _version=null, code=6690-2, _code=null, display=Leukocytes [#/volume] in Blood by Automated count, _display=null, userSelected=null, _userSelected=null}], text=Leukocytes [#/volume] in Blood by Automated count, _text=null}","{id=null, extension=null, reference=Patient/04c704c4-5d2d-4308-9c33-1690a6e47a6b, _reference=null, type=null, _type=null, identifier=null, display=null, _display=null}",,"{id=null, extension=null, reference=Encounter/0fce7a0a-3035-40c8-a129-50487fc75409, _reference=null, type=null, _type=null, identifier=null, display=null, _display=null}","2020-01-25T02:37:32-08:00",,,,,,"2020-01-25T02:37:32.457-08:00",,,"{id=null, extension=null, value=7, _value=null, comparator=null, _comparator=null, unit=10*3/uL, _unit=null, system=http://unitsofmeasure.org, _system=null, code=10*3/uL, _code=null}",,,,,,,,,,,,,,,,,,,,,,,,,,
"Observation","3dad82cc-b7f4-45a4-b6cf-0019db7825cb","{id=null, extension=null, versionId=null, _versionId=null, lastUpdated=2023-11-17T01:57:03.687Z, _lastUpdated=null, source=null, _source=null, profile=null, security=null, tag=null}",,,,,,,,,,,"final",,"[{id=null, extension=null, coding=[{id=null, extension=null, system=http://terminology.hl7.org/CodeSystem/observation-category, _system=null, version=null, _version=null, code=vital-signs, _code=null, display=vital-signs, _display=null, userSelected=null, _userSelected=null}], text=null, _text=null}]","{id=null, extension=null, coding=[{id=null, extension=null, system=http://loinc.org, _system=null, version=null, _version=null, code=2708-6, _code=null, display=Oxygen saturation in Arterial blood, _display=null, userSelected=null, _userSelected=null}, {id=null, extension=null, system=http://loinc.org, _system=null, version=null, _version=null, code=59408-5, _code=null, display=Oxygen saturation in Arterial blood by Pulse oximetry, _display=null, userSelected=null, _userSelected=null}], text=Oxygen saturation in Arterial blood, _text=null}","{id=null, extension=null, reference=Patient/04c704c4-5d2d-4308-9c33-1690a6e47a6b, _reference=null, type=null, _type=null, identifier=null, display=null, _display=null}",,"{id=null, extension=null, reference=Encounter/2349e4b5-5163-425d-a444-b0e459a0260d, _reference=null, type=null, _type=null, identifier=null, display=null, _display=null}","2020-03-10T03:37:32-07:00",,,,,,"2020-03-10T03:37:32.457-07:00",,,"{id=null, extension=null, value=82, _value=null, comparator=null, _comparator=null, unit=%, _unit=null, system=http://unitsofmeasure.org, _system=null, code=%, _code=null}",,,,,,,,,,,,,,,,,,,,,,,,,,
"Observation","a78f792c-03b9-41d7-b126-03cbbb9e93e8","{id=null, extension=null, versionId=null, _versionId=null, lastUpdated=2023-11-17T01:57:03.688Z, _lastUpdated=null, source=null, _source=null, profile=null, security=null, tag=null}",,,,,,,,,,,"final",,"[{id=null, extension=null, coding=[{id=null, extension=null, system=http://terminology.hl7.org/CodeSystem/observation-category, _system=null, version=null, _version=null, code=vital-signs, _code=null, display=vital-signs, _display=null, userSelected=null, _userSelected=null}], text=null, _text=null}]","{id=null, extension=null, coding=[{id=null, extension=null, system=http://loinc.org, _system=null, version=null, _version=null, code=29463-7, _code=null, display=Body Weight, _display=null, userSelected=null, _userSelected=null}], text=Body Weight, _text=null}","{id=null, extension=null, reference=Patient/04c704c4-5d2d-4308-9c33-1690a6e47a6b, _reference=null, type=null, _type=null, identifier=null, display=null, _display=null}",,"{id=null, extension=null, reference=Encounter/2349e4b5-5163-425d-a444-b0e459a0260d, _reference=null, type=null, _type=null, identifier=null, display=null, _display=null}","2020-03-10T03:37:32-07:00",,,,,,"2020-03-10T03:37:32.457-07:00",,,"{id=null, extension=null, value=80, _value=null, comparator=null, _comparator=null, unit=kg, _unit=null, system=http://unitsofmeasure.org, _system=null, code=kg, _code=null}",,,,,,,,,,,,,,,,,,,,,,,,,,
"Observation","7f2dec46-af49-4057-9cac-3e3663f9953c","{id=null, extension=null, versionId=null, _versionId=null, lastUpdated=2023-11-17T01:57:03.688Z, _lastUpdated=null, source=null, _source=null, profile=null, security=null, tag=null}",,,,,,,,,,,"final",,"[{id=null, extension=null, coding=[{id=null, extension=null, system=http://terminology.hl7.org/CodeSystem/observation-category, _system=null, version=null, _version=null, code=laboratory, _code=null, display=laboratory, _display=null, userSelected=null, _userSelected=null}], text=null, _text=null}]","{id=null, extension=null, coding=[{id=null, extension=null, system=http://loinc.org, _system=null, version=null, _version=null, code=786-4, _code=null, display=MCHC [Mass/volume] by Automated count, _display=null, userSelected=null, _userSelected=null}], text=MCHC [Mass/volume] by Automated count, _text=null}","{id=null, extension=null, reference=Patient/04c704c4-5d2d-4308-9c33-1690a6e47a6b, _reference=null, type=null, _type=null, identifier=null, display=null, _display=null}",,"{id=null, extension=null, reference=Encounter/0fce7a0a-3035-40c8-a129-50487fc75409, _reference=null, type=null, _type=null, identifier=null, display=null, _display=null}","2020-01-25T02:37:32-08:00",,,,,,"2020-01-25T02:37:32.457-08:00",,,"{id=null, extension=null, value=35, _value=null, comparator=null, _comparator=null, unit=g/dL, _unit=null, system=http://unitsofmeasure.org, _system=null, code=g/dL, _code=null}",,,,,,,,,,,,,,,,,,,,,,,,,,
"Observation","e9b7c651-2a07-4cc9-80de-762f34ba3416","{id=null, extension=null, versionId=null, _versionId=null, lastUpdated=2023-11-17T01:57:03.689Z, _lastUpdated=null, source=null, _source=null, profile=null, security=null, tag=null}",,,,,,,,,,,"final",,"[{id=null, extension=null, coding=[{id=null, extension=null, system=http://terminology.hl7.org/CodeSystem/observation-category, _system=null, version=null, _version=null, code=laboratory, _code=null, display=laboratory, _display=null, userSelected=null, _userSelected=null}], text=null, _text=null}]","{id=null, extension=null, coding=[{id=null, extension=null, system=http://loinc.org, _system=null, version=null, _version=null, code=80383-3, _code=null, display=Influenza virus B Ag [Presence] in Nasopharynx by Rapid immunoassay, _display=null, userSelected=null, _userSelected=null}], text=Influenza virus B Ag [Presence] in Nasopharynx by Rapid immunoassay, _text=null}","{id=null, extension=null, reference=Patient/04c704c4-5d2d-4308-9c33-1690a6e47a6b, _reference=null, type=null, _type=null, identifier=null, display=null, _display=null}",,"{id=null, extension=null, reference=Encounter/2349e4b5-5163-425d-a444-b0e459a0260d, _reference=null, type=null, _type=null, identifier=null, display=null, _display=null}","2020-03-10T03:37:32-07:00",,,,,,"2020-03-10T03:37:32.457-07:00",,,,"{id=null, extension=null, coding=[{id=null, extension=null, system=http://snomed.info/sct, _system=null, version=null, _version=null, code=260415000, _code=null, display=Not detected (qualifier value), _display=null, userSelected=null, _userSelected=null}], text=Not detected (qualifier value), _text=null}",,,,,,,,,,,,,,,,,,,,,,,,,
"Observation","a230eee5-06ea-44bf-b12a-752d89e5c816","{id=null, extension=null, versionId=null, _versionId=null, lastUpdated=2023-11-17T01:57:03.688Z, _lastUpdated=null, source=null, _source=null, profile=null, security=null, tag=null}",,,,,,,,,,,"final",,"[{id=null, extension=null, coding=[{id=null, extension=null, system=http://terminology.hl7.org/CodeSystem/observation-category, _system=null, version=null, _version=null, code=vital-signs, _code=null, display=vital-signs, _display=null, userSelected=null, _userSelected=null}], text=null, _text=null}]","{id=null, extension=null, coding=[{id=null, extension=null, system=http://loinc.org, _system=null, version=null, _version=null, code=29463-7, _code=null, display=Body Weight, _display=null, userSelected=null, _userSelected=null}], text=Body Weight, _text=null}","{id=null, extension=null, reference=Patient/04c704c4-5d2d-4308-9c33-1690a6e47a6b, _reference=null, type=null, _type=null, identifier=null, display=null, _display=null}",,"{id=null, extension=null, reference=Encounter/0fce7a0a-3035-40c8-a129-50487fc75409, _reference=null, type=null, _type=null, identifier=null, display=null, _display=null}","2020-01-25T02:37:32-08:00",,,,,,"2020-01-25T02:37:32.457-08:00",,,"{id=null, extension=null, value=80, _value=null, comparator=null, _comparator=null, unit=kg, _unit=null, system=http://unitsofmeasure.org, _system=null, code=kg, _code=null}",,,,,,,,,,,,,,,,,,,,,,,,,,
"Observation","31a6cff1-9756-4525-a67f-0b217b5fabcd","{id=null, extension=null, versionId=null, _versionId=null, lastUpdated=2023-11-17T01:57:03.687Z, _lastUpdated=null, source=null, _source=null, profile=null, security=null, tag=null}",,,,,,,,,,,"final",,"[{id=null, extension=null, coding=[{id=null, extension=null, system=http://terminology.hl7.org/CodeSystem/observation-category, _system=null, version=null, _version=null, code=vital-signs, _code=null, display=vital-signs, _display=null, userSelected=null, _userSelected=null}], text=null, _text=null}]","{id=null, extension=null, coding=[{id=null, extension=null, system=http://loinc.org, _system=null, version=null, _version=null, code=9279-1, _code=null, display=Respiratory rate, _display=null, userSelected=null, _userSelected=null}], text=Respiratory rate, _text=null}","{id=null, extension=null, reference=Patient/04c704c4-5d2d-4308-9c33-1690a6e47a6b, _reference=null, type=null, _type=null, identifier=null, display=null, _display=null}",,"{id=null, extension=null, reference=Encounter/0fce7a0a-3035-40c8-a129-50487fc75409, _reference=null, type=null, _type=null, identifier=null, display=null, _display=null}","2020-01-25T02:37:32-08:00",,,,,,"2020-01-25T02:37:32.457-08:00",,,"{id=null, extension=null, value=15, _value=null, comparator=null, _comparator=null, unit=/min, _unit=null, system=http://unitsofmeasure.org, _system=null, code=/min, _code=null}",,,,,,,,,,,,,,,,,,,,,,,,,,
"Observation","a415a5be-6ac0-4a0e-b9df-abd5f7104e9d","{id=null, extension=null, versionId=null, _versionId=null, lastUpdated=2023-11-17T01:57:03.688Z, _lastUpdated=null, source=null, _source=null, profile=null, security=null, tag=null}",,,,,,,,,,,"final",,"[{id=null, extension=null, coding=[{id=null, extension=null, system=http://terminology.hl7.org/CodeSystem/observation-category, _system=null, version=null, _version=null, code=laboratory, _code=null, display=laboratory, _display=null, userSelected=null, _userSelected=null}], text=null, _text=null}]","{id=null, extension=null, coding=[{id=null, extension=null, system=http://loinc.org, _system=null, version=null, _version=null, code=21000-5, _code=null, display=Erythrocyte distribution width [Entitic volume] by Automated count, _display=null, userSelected=null, _userSelected=null}], text=Erythrocyte distribution width [Entitic volume] by Automated count, _text=null}","{id=null, extension=null, reference=Patient/04c704c4-5d2d-4308-9c33-1690a6e47a6b, _reference=null, type=null, _type=null, identifier=null, display=null, _display=null}",,"{id=null, extension=null, reference=Encounter/0fce7a0a-3035-40c8-a129-50487fc75409, _reference=null, type=null, _type=null, identifier=null, display=null, _display=null}","2020-01-25T02:37:32-08:00",,,,,,"2020-01-25T02:37:32.457-08:00",,,"{id=null, extension=null, value=40, _value=null, comparator=null, _comparator=null, unit=fL, _unit=null, system=http://unitsofmeasure.org, _system=null, code=fL, _code=null}",,,,,,,,,,,,,,,,,,,,,,,,,,
"Observation","6a9c4a7c-341f-4d31-b4c9-56e92e9679f6","{id=null, extension=null, versionId=null, _versionId=null, lastUpdated=2023-11-17T01:57:03.688Z, _lastUpdated=null, source=null, _source=null, profile=null, security=null, tag=null}",,,,,,,,,,,"final",,"[{id=null, extension=null, coding=[{id=null, extension=null, system=http://terminology.hl7.org/CodeSystem/observation-category, _system=null, version=null, _version=null, code=vital-signs, _code=null, display=vital-signs, _display=null, userSelected=null, _userSelected=null}], text=null, _text=null}]","{id=null, extension=null, coding=[{id=null, extension=null, system=http://loinc.org, _system=null, version=null, _version=null, code=39156-5, _code=null, display=Body Mass Index, _display=null, userSelected=null, _userSelected=null}], text=Body Mass Index, _text=null}","{id=null, extension=null, reference=Patient/04c704c4-5d2d-4308-9c33-1690a6e47a6b, _reference=null, type=null, _type=null, identifier=null, display=null, _display=null}",,"{id=null, extension=null, reference=Encounter/0fce7a0a-3035-40c8-a129-50487fc75409, _reference=null, type=null, _type=null, identifier=null, display=null, _display=null}","2020-01-25T02:37:32-08:00",,,,,,"2020-01-25T02:37:32.457-08:00",,,"{id=null, extension=null, value=22, _value=null, comparator=null, _comparator=null, unit=kg/m2, _unit=null, system=http://unitsofmeasure.org, _system=null, code=kg/m2, _code=null}",,,,,,,,,,,,,,,,,,,,,,,,,,
"Observation","ed50596f-9ac0-49d2-83d3-765a4c489bda","{id=null, extension=null, versionId=null, _versionId=null, lastUpdated=2023-11-17T01:57:03.689Z, _lastUpdated=null, source=null, _source=null, profile=null, security=null, tag=null}",,,,,,,,,,,"final",,"[{id=null, extension=null, coding=[{id=null, extension=null, system=http://terminology.hl7.org/CodeSystem/observation-category, _system=null, version=null, _version=null, code=laboratory, _code=null, display=laboratory, _display=null, userSelected=null, _userSelected=null}], text=null, _text=null}]","{id=null, extension=null, coding=[{id=null, extension=null, system=http://loinc.org, _system=null, version=null, _version=null, code=4544-3, _code=null, display=Hematocrit [Volume Fraction] of Blood by Automated count, _display=null, userSelected=null, _userSelected=null}], text=Hematocrit [Volume Fraction] of Blood by Automated count, _text=null}","{id=null, extension=null, reference=Patient/04c704c4-5d2d-4308-9c33-1690a6e47a6b, _reference=null, type=null, _type=null, identifier=null, display=null, _display=null}",,"{id=null, extension=null, reference=Encounter/0fce7a0a-3035-40c8-a129-50487fc75409, _reference=null, type=null, _type=null, identifier=null, display=null, _display=null}","2020-01-25T02:37:32-08:00",,,,,,"2020-01-25T02:37:32.457-08:00",,,"{id=null, extension=null, value=36, _value=null, comparator=null, _comparator=null, unit=%, _unit=null, system=http://unitsofmeasure.org, _system=null, code=%, _code=null}",,,,,,,,,,,,,,,,,,,,,,,,,,
"Observation","42ed5a0c-3218-4727-9c77-0abd9b08988a","{id=null, extension=null, versionId=null, _versionId=null, lastUpdated=2023-11-17T01:57:03.687Z, _lastUpdated=null, source=null, _source=null, profile=null, security=null, tag=null}",,,,,,,,,,,"final",,"[{id=null, extension=null, coding=[{id=null, extension=null, system=http://terminology.hl7.org/CodeSystem/observation-category, _system=null, version=null, _version=null, code=vital-signs, _code=null, display=vital-signs, _display=null, userSelected=null, _userSelected=null}], text=null, _text=null}]","{id=null, extension=null, coding=[{id=null, extension=null, system=http://loinc.org, _system=null, version=null, _version=null, code=8302-2, _code=null, display=Body Height, _display=null, userSelected=null, _userSelected=null}], text=Body Height, _text=null}","{id=null, extension=null, reference=Patient/04c704c4-5d2d-4308-9c33-1690a6e47a6b, _reference=null, type=null, _type=null, identifier=null, display=null, _display=null}",,"{id=null, extension=null, reference=Encounter/0fce7a0a-3035-40c8-a129-50487fc75409, _reference=null, type=null, _type=null, identifier=null, display=null, _display=null}","2020-01-25T02:37:32-08:00",,,,,,"2020-01-25T02:37:32.457-08:00",,,"{id=null, extension=null, value=190, _value=null, comparator=null, _comparator=null, unit=cm, _unit=null, system=http://unitsofmeasure.org, _system=null, code=cm, _code=null}",,,,,,,,,,,,,,,,,,,,,,,,,,
"Observation","3b30c0ac-daff-44a1-a788-aa0f12d3a738","{id=null, extension=null, versionId=null, _versionId=null, lastUpdated=2023-11-17T01:57:03.687Z, _lastUpdated=null, source=null, _source=null, profile=null, security=null, tag=null}",,,,,,,,,,,"final",,"[{id=null, extension=null, coding=[{id=null, extension=null, system=http://terminology.hl7.org/CodeSystem/observation-category, _system=null, version=null, _version=null, code=laboratory, _code=null, display=laboratory, _display=null, userSelected=null, _userSelected=null}], text=null, _text=null}]","{id=null, extension=null, coding=[{id=null, extension=null, system=http://loinc.org, _system=null, version=null, _version=null, code=787-2, _code=null, display=MCV [Entitic volume] by Automated count, _display=null, userSelected=null, _userSelected=null}], text=MCV [Entitic volume] by Automated count, _text=null}","{id=null, extension=null, reference=Patient/04c704c4-5d2d-4308-9c33-1690a6e47a6b, _reference=null, type=null, _type=null, identifier=null, display=null, _display=null}",,"{id=null, extension=null, reference=Encounter/0fce7a0a-3035-40c8-a129-50487fc75409, _reference=null, type=null, _type=null, identifier=null, display=null, _display=null}","2020-01-25T02:37:32-08:00",,,,,,"2020-01-25T02:37:32.457-08:00",,,"{id=null, extension=null, value=93, _value=null, comparator=null, _comparator=null, unit=fL, _unit=null, system=http://unitsofmeasure.org, _system=null, code=fL, _code=null}",,,,,,,,,,,,,,,,,,,,,,,,,,
"Observation","3425ed4b-ccff-4a51-90c3-70fa70108a48","{id=null, extension=null, versionId=null, _versionId=null, lastUpdated=2023-11-17T01:57:03.687Z, _lastUpdated=null, source=null, _source=null, profile=null, security=null, tag=null}",,,,,,,,,,,"final",,"[{id=null, extension=null, coding=[{id=null, extension=null, system=http://terminology.hl7.org/CodeSystem/observation-category, _system=null, version=null, _version=null, code=vital-signs, _code=null, display=vital-signs, _display=null, userSelected=null, _userSelected=null}], text=null, _text=null}]","{id=null, extension=null, coding=[{id=null, extension=null, system=http://loinc.org, _system=null, version=null, _version=null, code=9279-1, _code=null, display=Respiratory rate, _display=null, userSelected=null, _userSelected=null}], text=Respiratory rate, _text=null}","{id=null, extension=null, reference=Patient/04c704c4-5d2d-4308-9c33-1690a6e47a6b, _reference=null, type=null, _type=null, identifier=null, display=null, _display=null}",,"{id=null, extension=null, reference=Encounter/2349e4b5-5163-425d-a444-b0e459a0260d, _reference=null, type=null, _type=null, identifier=null, display=null, _display=null}","2020-03-10T03:37:32-07:00",,,,,,"2020-03-10T03:37:32.457-07:00",,,"{id=null, extension=null, value=40, _value=null, comparator=null, _comparator=null, unit=/min, _unit=null, system=http://unitsofmeasure.org, _system=null, code=/min, _code=null}",,,,,,,,,,,,,,,,,,,,,,,,,,
"Observation","df68fb80-d56f-465a-a28a-1fc93856d63e","{id=null, extension=null, versionId=null, _versionId=null, lastUpdated=2023-11-17T01:57:03.689Z, _lastUpdated=null, source=null, _source=null, profile=null, security=null, tag=null}",,,,,,,,,,,"final",,"[{id=null, extension=null, coding=[{id=null, extension=null, system=http://terminology.hl7.org/CodeSystem/observation-category, _system=null, version=null, _version=null, code=vital-signs, _code=null, display=vital-signs, _display=null, userSelected=null, _userSelected=null}], text=null, _text=null}]","{id=null, extension=null, coding=[{id=null, extension=null, system=http://loinc.org, _system=null, version=null, _version=null, code=8310-5, _code=null, display=Body temperature, _display=null, userSelected=null, _userSelected=null}, {id=null, extension=null, system=http://loinc.org, _system=null, version=null, _version=null, code=8331-1, _code=null, display=Oral temperature, _display=null, userSelected=null, _userSelected=null}], text=Body temperature, _text=null}","{id=null, extension=null, reference=Patient/04c704c4-5d2d-4308-9c33-1690a6e47a6b, _reference=null, type=null, _type=null, identifier=null, display=null, _display=null}",,"{id=null, extension=null, reference=Encounter/2349e4b5-5163-425d-a444-b0e459a0260d, _reference=null, type=null, _type=null, identifier=null, display=null, _display=null}","2020-03-10T03:37:32-07:00",,,,,,"2020-03-10T03:37:32.457-07:00",,,"{id=null, extension=null, value=42, _value=null, comparator=null, _comparator=null, unit=Cel, _unit=null, system=http://unitsofmeasure.org, _system=null, code=Cel, _code=null}",,,,,,,,,,,,,,,,,,,,,,,,,,
"Observation","c242c989-a114-4109-bc54-cdb74715d7d0","{id=null, extension=null, versionId=null, _versionId=null, lastUpdated=2023-11-17T01:57:03.689Z, _lastUpdated=null, source=null, _source=null, profile=null, security=null, tag=null}",,,,,,,,,,,"final",,"[{id=null, extension=null, coding=[{id=null, extension=null, system=http://terminology.hl7.org/CodeSystem/observation-category, _system=null, version=null, _version=null, code=vital-signs, _code=null, display=vital-signs, _display=null, userSelected=null, _userSelected=null}], text=null, _text=null}]","{id=null, extension=null, coding=[{id=null, extension=null, system=http://loinc.org, _system=null, version=null, _version=null, code=85354-9, _code=null, display=Blood Pressure, _display=null, userSelected=null, _userSelected=null}], text=Blood Pressure, _text=null}","{id=null, extension=null, reference=Patient/04c704c4-5d2d-4308-9c33-1690a6e47a6b, _reference=null, type=null, _type=null, identifier=null, display=null, _display=null}",,"{id=null, extension=null, reference=Encounter/0fce7a0a-3035-40c8-a129-50487fc75409, _reference=null, type=null, _type=null, identifier=null, display=null, _display=null}","2020-01-25T02:37:32-08:00",,,,,,"2020-01-25T02:37:32.457-08:00",,,,,,,,,,,,,,,,,,,,,,,,,,,,,"[{id=null, extension=null, modifierExtension=null, code={id=null, extension=null, coding=[{id=null, extension=null, system=http://loinc.org, _system=null, version=null, _version=null, code=8462-4, _code=null, display=Diastolic Blood Pressure, _display=null, userSelected=null, _userSelected=null}], text=Diastolic Blood Pressure, _text=null}, valueQuantity={id=null, extension=null, value=87, _value=null, comparator=null, _comparator=null, unit=mm[Hg], _unit=null, system=http://unitsofmeasure.org, _system=null, code=mm[Hg], _code=null}, valueCodeableConcept=null, valueString=null, _valueString=null, valueBoolean=null, _valueBoolean=null, valueInteger=null, _valueInteger=null, valueRange=null, valueRatio=null, valueSampledData=null, valueTime=null, _valueTime=null, valueDateTime=null, _valueDateTime=null, valuePeriod=null, dataAbsentReason=null, interpretation=null, referenceRange=null}, {id=null, extension=null, modifierExtension=null, code={id=null, extension=null, coding=[{id=null, extension=null, system=http://loinc.org, _system=null, version=null, _version=null, code=8480-6, _code=null, display=Systolic Blood Pressure, _display=null, userSelected=null, _userSelected=null}], text=Systolic Blood Pressure, _text=null}, valueQuantity={id=null, extension=null, value=101, _value=null, comparator=null, _comparator=null, unit=mm[Hg], _unit=null, system=http://unitsofmeasure.org, _system=null, code=mm[Hg], _code=null}, valueCodeableConcept=null, valueString=null, _valueString=null, valueBoolean=null, _valueBoolean=null, valueInteger=null, _valueInteger=null, valueRange=null, valueRatio=null, valueSampledData=null, valueTime=null, _valueTime=null, valueDateTime=null, _valueDateTime=null, valuePeriod=null, dataAbsentReason=null, interpretation=null, referenceRange=null}]"
"Observation","79c989a4-9f91-4efb-abdf-fef70d8f5d50","{id=null, extension=null, versionId=null, _versionId=null, lastUpdated=2023-11-17T01:57:03.688Z, _lastUpdated=null, source=null, _source=null, profile=null, security=null, tag=null}",,,,,,,,,,,"final",,"[{id=null, extension=null, coding=[{id=null, extension=null, system=http://terminology.hl7.org/CodeSystem/observation-category, _system=null, version=null, _version=null, code=laboratory, _code=null, display=laboratory, _display=null, userSelected=null, _userSelected=null}], text=null, _text=null}]","{id=null, extension=null, coding=[{id=null, extension=null, system=http://loinc.org, _system=null, version=null, _version=null, code=718-7, _code=null, display=Hemoglobin [Mass/volume] in Blood, _display=null, userSelected=null, _userSelected=null}], text=Hemoglobin [Mass/volume] in Blood, _text=null}","{id=null, extension=null, reference=Patient/04c704c4-5d2d-4308-9c33-1690a6e47a6b, _reference=null, type=null, _type=null, identifier=null, display=null, _display=null}",,"{id=null, extension=null, reference=Encounter/0fce7a0a-3035-40c8-a129-50487fc75409, _reference=null, type=null, _type=null, identifier=null, display=null, _display=null}","2020-01-25T02:37:32-08:00",,,,,,"2020-01-25T02:37:32.457-08:00",,,"{id=null, extension=null, value=14, _value=null, comparator=null, _comparator=null, unit=g/dL, _unit=null, system=http://unitsofmeasure.org, _system=null, code=g/dL, _code=null}",,,,,,,,,,,,,,,,,,,,,,,,,,
"Observation","6d56d809-1e9b-417a-a946-a546037670c2","{id=null, extension=null, versionId=null, _versionId=null, lastUpdated=2023-11-17T01:57:03.688Z, _lastUpdated=null, source=null, _source=null, profile=null, security=null, tag=null}",,,,,,,,,,,"final",,"[{id=null, extension=null, coding=[{id=null, extension=null, system=http://terminology.hl7.org/CodeSystem/observation-category, _system=null, version=null, _version=null, code=vital-signs, _code=null, display=vital-signs, _display=null, userSelected=null, _userSelected=null}], text=null, _text=null}]","{id=null, extension=null, coding=[{id=null, extension=null, system=http://loinc.org, _system=null, version=null, _version=null, code=85354-9, _code=null, display=Blood Pressure, _display=null, userSelected=null, _userSelected=null}], text=Blood Pressure, _text=null}","{id=null, extension=null, reference=Patient/04c704c4-5d2d-4308-9c33-1690a6e47a6b, _reference=null, type=null, _type=null, identifier=null, display=null, _display=null}",,"{id=null, extension=null, reference=Encounter/2349e4b5-5163-425d-a444-b0e459a0260d, _reference=null, type=null, _type=null, identifier=null, display=null, _display=null}","2020-03-10T03:37:32-07:00",,,,,,"2020-03-10T03:37:32.457-07:00",,,,,,,,,,,,,,,,,,,,,,,,,,,,,"[{id=null, extension=null, modifierExtension=null, code={id=null, extension=null, coding=[{id=null, extension=null, system=http://loinc.org, _system=null, version=null, _version=null, code=8462-4, _code=null, display=Diastolic Blood Pressure, _display=null, userSelected=null, _userSelected=null}], text=Diastolic Blood Pressure, _text=null}, valueQuantity={id=null, extension=null, value=87, _value=null, comparator=null, _comparator=null, unit=mm[Hg], _unit=null, system=http://unitsofmeasure.org, _system=null, code=mm[Hg], _code=null}, valueCodeableConcept=null, valueString=null, _valueString=null, valueBoolean=null, _valueBoolean=null, valueInteger=null, _valueInteger=null, valueRange=null, valueRatio=null, valueSampledData=null, valueTime=null, _valueTime=null, valueDateTime=null, _valueDateTime=null, valuePeriod=null, dataAbsentReason=null, interpretation=null, referenceRange=null}, {id=null, extension=null, modifierExtension=null, code={id=null, extension=null, coding=[{id=null, extension=null, system=http://loinc.org, _system=null, version=null, _version=null, code=8480-6, _code=null, display=Systolic Blood Pressure, _display=null, userSelected=null, _userSelected=null}], text=Systolic Blood Pressure, _text=null}, valueQuantity={id=null, extension=null, value=130, _value=null, comparator=null, _comparator=null, unit=mm[Hg], _unit=null, system=http://unitsofmeasure.org, _system=null, code=mm[Hg], _code=null}, valueCodeableConcept=null, valueString=null, _valueString=null, valueBoolean=null, _valueBoolean=null, valueInteger=null, _valueInteger=null, valueRange=null, valueRatio=null, valueSampledData=null, valueTime=null, _valueTime=null, valueDateTime=null, _valueDateTime=null, valuePeriod=null, dataAbsentReason=null, interpretation=null, referenceRange=null}]"
"Observation","dc2a1f7a-3d46-4644-b4cc-cca9232fe62b","{id=null, extension=null, versionId=null, _versionId=null, lastUpdated=2023-11-17T01:57:03.689Z, _lastUpdated=null, source=null, _source=null, profile=null, security=null, tag=null}",,,,,,,,,,,"final",,"[{id=null, extension=null, coding=[{id=null, extension=null, system=http://terminology.hl7.org/CodeSystem/observation-category, _system=null, version=null, _version=null, code=vital-signs, _code=null, display=vital-signs, _display=null, userSelected=null, _userSelected=null}], text=null, _text=null}]","{id=null, extension=null, coding=[{id=null, extension=null, system=http://loinc.org, _system=null, version=null, _version=null, code=8867-4, _code=null, display=Heart rate, _display=null, userSelected=null, _userSelected=null}], text=Heart rate, _text=null}","{id=null, extension=null, reference=Patient/04c704c4-5d2d-4308-9c33-1690a6e47a6b, _reference=null, type=null, _type=null, identifier=null, display=null, _display=null}",,"{id=null, extension=null, reference=Encounter/0fce7a0a-3035-40c8-a129-50487fc75409, _reference=null, type=null, _type=null, identifier=null, display=null, _display=null}","2020-01-25T02:37:32-08:00",,,,,,"2020-01-25T02:37:32.457-08:00",,,"{id=null, extension=null, value=82, _value=null, comparator=null, _comparator=null, unit=/min, _unit=null, system=http://unitsofmeasure.org, _system=null, code=/min, _code=null}",,,,,,,,,,,,,,,,,,,,,,,,,,
"Observation","765d2abe-5a40-4743-8946-b70d0db30c8f","{id=null, extension=null, versionId=null, _versionId=null, lastUpdated=2023-11-17T01:57:03.688Z, _lastUpdated=null, source=null, _source=null, profile=null, security=null, tag=null}",,,,,,,,,,,"final",,"[{id=null, extension=null, coding=[{id=null, extension=null, system=http://terminology.hl7.org/CodeSystem/observation-category, _system=null, version=null, _version=null, code=laboratory, _code=null, display=laboratory, _display=null, userSelected=null, _userSelected=null}], text=null, _text=null}]","{id=null, extension=null, coding=[{id=null, extension=null, system=http://loinc.org, _system=null, version=null, _version=null, code=80382-5, _code=null, display=Influenza virus A Ag [Presence] in Nasopharynx by Rapid immunoassay, _display=null, userSelected=null, _userSelected=null}], text=Influenza virus A Ag [Presence] in Nasopharynx by Rapid immunoassay, _text=null}","{id=null, extension=null, reference=Patient/04c704c4-5d2d-4308-9c33-1690a6e47a6b, _reference=null, type=null, _type=null, identifier=null, display=null, _display=null}",,"{id=null, extension=null, reference=Encounter/2349e4b5-5163-425d-a444-b0e459a0260d, _reference=null, type=null, _type=null, identifier=null, display=null, _display=null}","2020-03-10T03:37:32-07:00",,,,,,"2020-03-10T03:37:32.457-07:00",,,,"{id=null, extension=null, coding=[{id=null, extension=null, system=http://snomed.info/sct, _system=null, version=null, _version=null, code=260415000, _code=null, display=Not detected (qualifier value), _display=null, userSelected=null, _userSelected=null}], text=Not detected (qualifier value), _text=null}",,,,,,,,,,,,,,,,,,,,,,,,,
"Observation","6c662353-8856-4e87-bc06-c3b6deb9b85b","{id=null, extension=null, versionId=null, _versionId=null, lastUpdated=2023-11-17T01:57:03.688Z, _lastUpdated=null, source=null, _source=null, profile=null, security=null, tag=null}",,,,,,,,,,,"final",,"[{id=null, extension=null, coding=[{id=null, extension=null, system=http://terminology.hl7.org/CodeSystem/observation-category, _system=null, version=null, _version=null, code=survey, _code=null, display=survey, _display=null, userSelected=null, _userSelected=null}], text=null, _text=null}]","{id=null, extension=null, coding=[{id=null, extension=null, system=http://loinc.org, _system=null, version=null, _version=null, code=72166-2, _code=null, display=Tobacco smoking status NHIS, _display=null, userSelected=null, _userSelected=null}], text=Tobacco smoking status NHIS, _text=null}","{id=null, extension=null, reference=Patient/04c704c4-5d2d-4308-9c33-1690a6e47a6b, _reference=null, type=null, _type=null, identifier=null, display=null, _display=null}",,"{id=null, extension=null, reference=Encounter/0fce7a0a-3035-40c8-a129-50487fc75409, _reference=null, type=null, _type=null, identifier=null, display=null, _display=null}","2020-01-25T02:37:32-08:00",,,,,,"2020-01-25T02:37:32.457-08:00",,,,"{id=null, extension=null, coding=[{id=null, extension=null, system=http://snomed.info/sct, _system=null, version=null, _version=null, code=266919005, _code=null, display=Never smoker, _display=null, userSelected=null, _userSelected=null}], text=Never smoker, _text=null}",,,,,,,,,,,,,,,,,,,,,,,,,
"Observation","aa50a583-02a4-490f-93f0-b34ed88f312e","{id=null, extension=null, versionId=null, _versionId=null, lastUpdated=2023-11-17T01:57:03.688Z, _lastUpdated=null, source=null, _source=null, profile=null, security=null, tag=null}",,,,,,,,,,,"final",,"[{id=null, extension=null, coding=[{id=null, extension=null, system=http://terminology.hl7.org/CodeSystem/observation-category, _system=null, version=null, _version=null, code=laboratory, _code=null, display=laboratory, _display=null, userSelected=null, _userSelected=null}], text=null, _text=null}]","{id=null, extension=null, coding=[{id=null, extension=null, system=http://loinc.org, _system=null, version=null, _version=null, code=32623-1, _code=null, display=Platelet mean volume [Entitic volume] in Blood by Automated count, _display=null, userSelected=null, _userSelected=null}], text=Platelet mean volume [Entitic volume] in Blood by Automated count, _text=null}","{id=null, extension=null, reference=Patient/04c704c4-5d2d-4308-9c33-1690a6e47a6b, _reference=null, type=null, _type=null, identifier=null, display=null, _display=null}",,"{id=null, extension=null, reference=Encounter/0fce7a0a-3035-40c8-a129-50487fc75409, _reference=null, type=null, _type=null, identifier=null, display=null, _display=null}","2020-01-25T02:37:32-08:00",,,,,,"2020-01-25T02:37:32.457-08:00",,,"{id=null, extension=null, value=11, _value=null, comparator=null, _comparator=null, unit=fL, _unit=null, system=http://unitsofmeasure.org, _system=null, code=fL, _code=null}",,,,,,,,,,,,,,,,,,,,,,,,,,
"Observation","67473132-b746-45ed-ac17-967f1eb8b6d7","{id=null, extension=null, versionId=null, _versionId=null, lastUpdated=2023-11-17T01:57:03.688Z, _lastUpdated=null, source=null, _source=null, profile=null, security=null, tag=null}",,,,,,,,,,,"final",,"[{id=null, extension=null, coding=[{id=null, extension=null, system=http://terminology.hl7.org/CodeSystem/observation-category, _system=null, version=null, _version=null, code=laboratory, _code=null, display=laboratory, _display=null, userSelected=null, _userSelected=null}], text=null, _text=null}]","{id=null, extension=null, coding=[{id=null, extension=null, system=http://loinc.org, _system=null, version=null, _version=null, code=94531-1, _code=null, display=SARS-CoV-2 RNA Pnl Resp NAA+probe, _display=null, userSelected=null, _userSelected=null}], text=SARS-CoV-2 RNA Pnl Resp NAA+probe, _text=null}","{id=null, extension=null, reference=Patient/04c704c4-5d2d-4308-9c33-1690a6e47a6b, _reference=null, type=null, _type=null, identifier=null, display=null, _display=null}",,"{id=null, extension=null, reference=Encounter/2349e4b5-5163-425d-a444-b0e459a0260d, _reference=null, type=null, _type=null, identifier=null, display=null, _display=null}","2020-03-10T05:00:32-07:00",,,,,,"2020-03-10T05:00:32.457-07:00",,,,"{id=null, extension=null, coding=[{id=null, extension=null, system=http://snomed.info/sct, _system=null, version=null, _version=null, code=260373001, _code=null, display=Detected (qualifier value), _display=null, userSelected=null, _userSelected=null}], text=Detected (qualifier value), _text=null}",,,,,,,,,,,,,,,,,,,,,,,,,
"Observation","ca51d627-46a7-461a-a9f1-1f8cfa07a0d0","{id=null, extension=null, versionId=null, _versionId=null, lastUpdated=2023-11-17T01:57:03.689Z, _lastUpdated=null, source=null, _source=null, profile=null, security=null, tag=null}",,,,,,,,,,,"final",,"[{id=null, extension=null, coding=[{id=null, extension=null, system=http://terminology.hl7.org/CodeSystem/observation-category, _system=null, version=null, _version=null, code=vital-signs, _code=null, display=vital-signs, _display=null, userSelected=null, _userSelected=null}], text=null, _text=null}]","{id=null, extension=null, coding=[{id=null, extension=null, system=http://loinc.org, _system=null, version=null, _version=null, code=8310-5, _code=null, display=Body temperature, _display=null, userSelected=null, _userSelected=null}, {id=null, extension=null, system=http://loinc.org, _system=null, version=null, _version=null, code=8331-1, _code=null, display=Oral temperature, _display=null, userSelected=null, _userSelected=null}], text=Body temperature, _text=null}","{id=null, extension=null, reference=Patient/04c704c4-5d2d-4308-9c33-1690a6e47a6b, _reference=null, type=null, _type=null, identifier=null, display=null, _display=null}",,"{id=null, extension=null, reference=Encounter/dc3fa5ac-efdb-43e0-adad-b38abfc94458, _reference=null, type=null, _type=null, identifier=null, display=null, _display=null}","2018-10-06T03:37:32-07:00",,,,,,"2018-10-06T03:37:32.457-07:00",,,"{id=null, extension=null, value=38, _value=null, comparator=null, _comparator=null, unit=Cel, _unit=null, system=http://unitsofmeasure.org, _system=null, code=Cel, _code=null}",,,,,,,,,,,,,,,,,,,,,,,,,,
"Observation","15f60857-22e1-4219-977f-53802fcbc771","{id=null, extension=null, versionId=null, _versionId=null, lastUpdated=2023-11-17T01:57:03.687Z, _lastUpdated=null, source=null, _source=null, profile=null, security=null, tag=null}",,,,,,,,,,,"final",,"[{id=null, extension=null, coding=[{id=null, extension=null, system=http://terminology.hl7.org/CodeSystem/observation-category, _system=null, version=null, _version=null, code=laboratory, _code=null, display=laboratory, _display=null, userSelected=null, _userSelected=null}], text=null, _text=null}]","{id=null, extension=null, coding=[{id=null, extension=null, system=http://loinc.org, _system=null, version=null, _version=null, code=32207-3, _code=null, display=Platelet distribution width [Entitic volume] in Blood by Automated count, _display=null, userSelected=null, _userSelected=null}], text=Platelet distribution width [Entitic volume] in Blood by Automated count, _text=null}","{id=null, extension=null, reference=Patient/04c704c4-5d2d-4308-9c33-1690a6e47a6b, _reference=null, type=null, _type=null, identifier=null, display=null, _display=null}",,"{id=null, extension=null, reference=Encounter/0fce7a0a-3035-40c8-a129-50487fc75409, _reference=null, type=null, _type=null, identifier=null, display=null, _display=null}","2020-01-25T02:37:32-08:00",,,,,,"2020-01-25T02:37:32.457-08:00",,,"{id=null, extension=null, value=294, _value=null, comparator=null, _comparator=null, unit=fL, _unit=null, system=http://unitsofmeasure.org, _system=null, code=fL, _code=null}",,,,,,,,,,,,,,,,,,,,,,,,,,
"Observation","f4243611-79f5-4dec-b9f2-a1b45f02c15e","{id=null, extension=null, versionId=null, _versionId=null, lastUpdated=2023-11-17T01:57:03.689Z, _lastUpdated=null, source=null, _source=null, profile=null, security=null, tag=null}",,,,,,,,,,,"final",,"[{id=null, extension=null, coding=[{id=null, extension=null, system=http://terminology.hl7.org/CodeSystem/observation-category, _system=null, version=null, _version=null, code=laboratory, _code=null, display=laboratory, _display=null, userSelected=null, _userSelected=null}], text=null, _text=null}]","{id=null, extension=null, coding=[{id=null, extension=null, system=http://loinc.org, _system=null, version=null, _version=null, code=777-3, _code=null, display=Platelets [#/volume] in Blood by Automated count, _display=null, userSelected=null, _userSelected=null}], text=Platelets [#/volume] in Blood by Automated count, _text=null}","{id=null, extension=null, reference=Patient/04c704c4-5d2d-4308-9c33-1690a6e47a6b, _reference=null, type=null, _type=null, identifier=null, display=null, _display=null}",,"{id=null, extension=null, reference=Encounter/0fce7a0a-3035-40c8-a129-50487fc75409, _reference=null, type=null, _type=null, identifier=null, display=null, _display=null}","2020-01-25T02:37:32-08:00",,,,,,"2020-01-25T02:37:32.457-08:00",,,"{id=null, extension=null, value=363, _value=null, comparator=null, _comparator=null, unit=10*3/uL, _unit=null, system=http://unitsofmeasure.org, _system=null, code=10*3/uL, _code=null}",,,,,,,,,,,,,,,,,,,,,,,,,,
"Observation","12b9e914-0fe5-48f2-bd3a-b2654833c555","{id=null, extension=null, versionId=null, _versionId=null, lastUpdated=2023-11-17T01:57:03.687Z, _lastUpdated=null, source=null, _source=null, profile=null, security=null, tag=null}",,,,,,,,,,,"final",,"[{id=null, extension=null, coding=[{id=null, extension=null, system=http://terminology.hl7.org/CodeSystem/observation-category, _system=null, version=null, _version=null, code=laboratory, _code=null, display=laboratory, _display=null, userSelected=null, _userSelected=null}], text=null, _text=null}]","{id=null, extension=null, coding=[{id=null, extension=null, system=http://loinc.org, _system=null, version=null, _version=null, code=789-8, _code=null, display=Erythrocytes [#/volume] in Blood by Automated count, _display=null, userSelected=null, _userSelected=null}], text=Erythrocytes [#/volume] in Blood by Automated count, _text=null}","{id=null, extension=null, reference=Patient/04c704c4-5d2d-4308-9c33-1690a6e47a6b, _reference=null, type=null, _type=null, identifier=null, display=null, _display=null}",,"{id=null, extension=null, reference=Encounter/0fce7a0a-3035-40c8-a129-50487fc75409, _reference=null, type=null, _type=null, identifier=null, display=null, _display=null}","2020-01-25T02:37:32-08:00",,,,,,"2020-01-25T02:37:32.457-08:00",,,"{id=null, extension=null, value=5, _value=null, comparator=null, _comparator=null, unit=10*6/uL, _unit=null, system=http://unitsofmeasure.org, _system=null, code=10*6/uL, _code=null}",,,,,,,,,,,,,,,,,,,,,,,,,,
"""
sample_patient_conditions = """
"resourcetype","id","meta","implicitrules","_implicitrules","language","_language","text","extension","modifierextension","identifier","clinicalstatus","verificationstatus","category","severity","code","bodysite","subject","encounter","onsetdatetime","_onsetdatetime","onsetage","onsetperiod","onsetrange","onsetstring","_onsetstring","abatementdatetime","_abatementdatetime","abatementage","abatementperiod","abatementrange","abatementstring","_abatementstring","recordeddate","_recordeddate","recorder","asserter","stage","evidence","note"
"Condition","9cc387ae-ba2f-4fcf-8e44-76e3031c1b43","{id=null, extension=null, versionId=null, _versionId=null, lastUpdated=2023-11-17T01:57:03.688Z, _lastUpdated=null, source=null, _source=null, profile=null, security=null, tag=null}",,,,,,,,,"{id=null, extension=null, coding=[{id=null, extension=null, system=http://terminology.hl7.org/CodeSystem/condition-clinical, _system=null, version=null, _version=null, code=resolved, _code=null, display=null, _display=null, userSelected=null, _userSelected=null}], text=null, _text=null}","{id=null, extension=null, coding=[{id=null, extension=null, system=http://terminology.hl7.org/CodeSystem/condition-ver-status, _system=null, version=null, _version=null, code=confirmed, _code=null, display=null, _display=null, userSelected=null, _userSelected=null}], text=null, _text=null}",,,"{id=null, extension=null, coding=[{id=null, extension=null, system=http://snomed.info/sct, _system=null, version=null, _version=null, code=25064002, _code=null, display=Headache (finding), _display=null, userSelected=null, _userSelected=null}], text=Headache (finding), _text=null}",,"{id=null, extension=null, reference=Patient/04c704c4-5d2d-4308-9c33-1690a6e47a6b, _reference=null, type=null, _type=null, identifier=null, display=null, _display=null}","{id=null, extension=null, reference=Encounter/2349e4b5-5163-425d-a444-b0e459a0260d, _reference=null, type=null, _type=null, identifier=null, display=null, _display=null}","2020-03-10T03:37:32-07:00",,,,,,,"2020-04-02T05:00:32-07:00",,,,,,,"2020-03-10T03:37:32-07:00",,,,,,
"Condition","1ab272d2-a36f-4ef8-9cd9-680153a8e704","{id=null, extension=null, versionId=null, _versionId=null, lastUpdated=2023-11-17T01:57:03.687Z, _lastUpdated=null, source=null, _source=null, profile=null, security=null, tag=null}",,,,,,,,,"{id=null, extension=null, coding=[{id=null, extension=null, system=http://terminology.hl7.org/CodeSystem/condition-clinical, _system=null, version=null, _version=null, code=resolved, _code=null, display=null, _display=null, userSelected=null, _userSelected=null}], text=null, _text=null}","{id=null, extension=null, coding=[{id=null, extension=null, system=http://terminology.hl7.org/CodeSystem/condition-ver-status, _system=null, version=null, _version=null, code=confirmed, _code=null, display=null, _display=null, userSelected=null, _userSelected=null}], text=null, _text=null}",,,"{id=null, extension=null, coding=[{id=null, extension=null, system=http://snomed.info/sct, _system=null, version=null, _version=null, code=267036007, _code=null, display=Dyspnea (finding), _display=null, userSelected=null, _userSelected=null}], text=Dyspnea (finding), _text=null}",,"{id=null, extension=null, reference=Patient/04c704c4-5d2d-4308-9c33-1690a6e47a6b, _reference=null, type=null, _type=null, identifier=null, display=null, _display=null}","{id=null, extension=null, reference=Encounter/2349e4b5-5163-425d-a444-b0e459a0260d, _reference=null, type=null, _type=null, identifier=null, display=null, _display=null}","2020-03-10T03:37:32-07:00",,,,,,,"2020-04-02T05:00:32-07:00",,,,,,,"2020-03-10T03:37:32-07:00",,,,,,
"Condition","bc1c3b7b-7a0c-417d-b12e-0062bf0bd4b4","{id=null, extension=null, versionId=null, _versionId=null, lastUpdated=2023-11-17T01:57:03.689Z, _lastUpdated=null, source=null, _source=null, profile=null, security=null, tag=null}",,,,,,,,,"{id=null, extension=null, coding=[{id=null, extension=null, system=http://terminology.hl7.org/CodeSystem/condition-clinical, _system=null, version=null, _version=null, code=resolved, _code=null, display=null, _display=null, userSelected=null, _userSelected=null}], text=null, _text=null}","{id=null, extension=null, coding=[{id=null, extension=null, system=http://terminology.hl7.org/CodeSystem/condition-ver-status, _system=null, version=null, _version=null, code=confirmed, _code=null, display=null, _display=null, userSelected=null, _userSelected=null}], text=null, _text=null}",,,"{id=null, extension=null, coding=[{id=null, extension=null, system=http://snomed.info/sct, _system=null, version=null, _version=null, code=840544004, _code=null, display=Suspected COVID-19, _display=null, userSelected=null, _userSelected=null}], text=Suspected COVID-19, _text=null}",,"{id=null, extension=null, reference=Patient/04c704c4-5d2d-4308-9c33-1690a6e47a6b, _reference=null, type=null, _type=null, identifier=null, display=null, _display=null}","{id=null, extension=null, reference=Encounter/2349e4b5-5163-425d-a444-b0e459a0260d, _reference=null, type=null, _type=null, identifier=null, display=null, _display=null}","2020-03-10T03:37:32-07:00",,,,,,,"2020-03-10T05:00:32-07:00",,,,,,,"2020-03-10T03:37:32-07:00",,,,,,
"Condition","7d03ddae-d49a-4a96-b9b9-b5dae9a87d2d","{id=null, extension=null, versionId=null, _versionId=null, lastUpdated=2023-11-17T01:57:03.688Z, _lastUpdated=null, source=null, _source=null, profile=null, security=null, tag=null}",,,,,,,,,"{id=null, extension=null, coding=[{id=null, extension=null, system=http://terminology.hl7.org/CodeSystem/condition-clinical, _system=null, version=null, _version=null, code=resolved, _code=null, display=null, _display=null, userSelected=null, _userSelected=null}], text=null, _text=null}","{id=null, extension=null, coding=[{id=null, extension=null, system=http://terminology.hl7.org/CodeSystem/condition-ver-status, _system=null, version=null, _version=null, code=confirmed, _code=null, display=null, _display=null, userSelected=null, _userSelected=null}], text=null, _text=null}",,,"{id=null, extension=null, coding=[{id=null, extension=null, system=http://snomed.info/sct, _system=null, version=null, _version=null, code=386661006, _code=null, display=Fever (finding), _display=null, userSelected=null, _userSelected=null}], text=Fever (finding), _text=null}",,"{id=null, extension=null, reference=Patient/04c704c4-5d2d-4308-9c33-1690a6e47a6b, _reference=null, type=null, _type=null, identifier=null, display=null, _display=null}","{id=null, extension=null, reference=Encounter/2349e4b5-5163-425d-a444-b0e459a0260d, _reference=null, type=null, _type=null, identifier=null, display=null, _display=null}","2020-03-10T03:37:32-07:00",,,,,,,"2020-04-02T05:00:32-07:00",,,,,,,"2020-03-10T03:37:32-07:00",,,,,,
"Condition","3edd1b2c-4743-4fc6-b3db-bf469c983061","{id=null, extension=null, versionId=null, _versionId=null, lastUpdated=2023-11-17T01:57:03.687Z, _lastUpdated=null, source=null, _source=null, profile=null, security=null, tag=null}",,,,,,,,,"{id=null, extension=null, coding=[{id=null, extension=null, system=http://terminology.hl7.org/CodeSystem/condition-clinical, _system=null, version=null, _version=null, code=resolved, _code=null, display=null, _display=null, userSelected=null, _userSelected=null}], text=null, _text=null}","{id=null, extension=null, coding=[{id=null, extension=null, system=http://terminology.hl7.org/CodeSystem/condition-ver-status, _system=null, version=null, _version=null, code=confirmed, _code=null, display=null, _display=null, userSelected=null, _userSelected=null}], text=null, _text=null}",,,"{id=null, extension=null, coding=[{id=null, extension=null, system=http://snomed.info/sct, _system=null, version=null, _version=null, code=43878008, _code=null, display=Streptococcal sore throat (disorder), _display=null, userSelected=null, _userSelected=null}], text=Streptococcal sore throat (disorder), _text=null}",,"{id=null, extension=null, reference=Patient/04c704c4-5d2d-4308-9c33-1690a6e47a6b, _reference=null, type=null, _type=null, identifier=null, display=null, _display=null}","{id=null, extension=null, reference=Encounter/dc3fa5ac-efdb-43e0-adad-b38abfc94458, _reference=null, type=null, _type=null, identifier=null, display=null, _display=null}","2018-10-06T03:37:32-07:00",,,,,,,"2018-10-15T03:37:32-07:00",,,,,,,"2018-10-06T03:37:32-07:00",,,,,,
"Condition","14644598-c2d0-4526-a880-66a3154d9f58","{id=null, extension=null, versionId=null, _versionId=null, lastUpdated=2023-11-17T01:57:03.687Z, _lastUpdated=null, source=null, _source=null, profile=null, security=null, tag=null}",,,,,,,,,"{id=null, extension=null, coding=[{id=null, extension=null, system=http://terminology.hl7.org/CodeSystem/condition-clinical, _system=null, version=null, _version=null, code=resolved, _code=null, display=null, _display=null, userSelected=null, _userSelected=null}], text=null, _text=null}","{id=null, extension=null, coding=[{id=null, extension=null, system=http://terminology.hl7.org/CodeSystem/condition-ver-status, _system=null, version=null, _version=null, code=confirmed, _code=null, display=null, _display=null, userSelected=null, _userSelected=null}], text=null, _text=null}",,,"{id=null, extension=null, coding=[{id=null, extension=null, system=http://snomed.info/sct, _system=null, version=null, _version=null, code=49727002, _code=null, display=Cough (finding), _display=null, userSelected=null, _userSelected=null}], text=Cough (finding), _text=null}",,"{id=null, extension=null, reference=Patient/04c704c4-5d2d-4308-9c33-1690a6e47a6b, _reference=null, type=null, _type=null, identifier=null, display=null, _display=null}","{id=null, extension=null, reference=Encounter/2349e4b5-5163-425d-a444-b0e459a0260d, _reference=null, type=null, _type=null, identifier=null, display=null, _display=null}","2020-03-10T03:37:32-07:00",,,,,,,"2020-04-02T05:00:32-07:00",,,,,,,"2020-03-10T03:37:32-07:00",,,,,,
"Condition","500d6754-1856-46f9-a007-4150199e3441","{id=null, extension=null, versionId=null, _versionId=null, lastUpdated=2023-11-17T01:57:03.687Z, _lastUpdated=null, source=null, _source=null, profile=null, security=null, tag=null}",,,,,,,,,"{id=null, extension=null, coding=[{id=null, extension=null, system=http://terminology.hl7.org/CodeSystem/condition-clinical, _system=null, version=null, _version=null, code=resolved, _code=null, display=null, _display=null, userSelected=null, _userSelected=null}], text=null, _text=null}","{id=null, extension=null, coding=[{id=null, extension=null, system=http://terminology.hl7.org/CodeSystem/condition-ver-status, _system=null, version=null, _version=null, code=confirmed, _code=null, display=null, _display=null, userSelected=null, _userSelected=null}], text=null, _text=null}",,,"{id=null, extension=null, coding=[{id=null, extension=null, system=http://snomed.info/sct, _system=null, version=null, _version=null, code=56018004, _code=null, display=Wheezing (finding), _display=null, userSelected=null, _userSelected=null}], text=Wheezing (finding), _text=null}",,"{id=null, extension=null, reference=Patient/04c704c4-5d2d-4308-9c33-1690a6e47a6b, _reference=null, type=null, _type=null, identifier=null, display=null, _display=null}","{id=null, extension=null, reference=Encounter/2349e4b5-5163-425d-a444-b0e459a0260d, _reference=null, type=null, _type=null, identifier=null, display=null, _display=null}","2020-03-10T03:37:32-07:00",,,,,,,"2020-04-02T05:00:32-07:00",,,,,,,"2020-03-10T03:37:32-07:00",,,,,,
"Condition","1d4e1d16-d2ee-4fdf-925a-fff225d1035d","{id=null, extension=null, versionId=null, _versionId=null, lastUpdated=2023-11-17T01:57:03.687Z, _lastUpdated=null, source=null, _source=null, profile=null, security=null, tag=null}",,,,,,,,,"{id=null, extension=null, coding=[{id=null, extension=null, system=http://terminology.hl7.org/CodeSystem/condition-clinical, _system=null, version=null, _version=null, code=resolved, _code=null, display=null, _display=null, userSelected=null, _userSelected=null}], text=null, _text=null}","{id=null, extension=null, coding=[{id=null, extension=null, system=http://terminology.hl7.org/CodeSystem/condition-ver-status, _system=null, version=null, _version=null, code=confirmed, _code=null, display=null, _display=null, userSelected=null, _userSelected=null}], text=null, _text=null}",,,"{id=null, extension=null, coding=[{id=null, extension=null, system=http://snomed.info/sct, _system=null, version=null, _version=null, code=444814009, _code=null, display=Viral sinusitis (disorder), _display=null, userSelected=null, _userSelected=null}], text=Viral sinusitis (disorder), _text=null}",,"{id=null, extension=null, reference=Patient/04c704c4-5d2d-4308-9c33-1690a6e47a6b, _reference=null, type=null, _type=null, identifier=null, display=null, _display=null}","{id=null, extension=null, reference=Encounter/cb5a9c67-6e54-49de-b5ad-c733cf348913, _reference=null, type=null, _type=null, identifier=null, display=null, _display=null}","2018-02-19T02:37:32-08:00",,,,,,,"2018-03-12T03:37:32-07:00",,,,,,,"2018-02-19T02:37:32-08:00",,,,,,
"Condition","0fb38c67-87dd-40e1-b950-f65d1a91ba85","{id=null, extension=null, versionId=null, _versionId=null, lastUpdated=2023-11-17T01:57:03.687Z, _lastUpdated=null, source=null, _source=null, profile=null, security=null, tag=null}",,,,,,,,,"{id=null, extension=null, coding=[{id=null, extension=null, system=http://terminology.hl7.org/CodeSystem/condition-clinical, _system=null, version=null, _version=null, code=resolved, _code=null, display=null, _display=null, userSelected=null, _userSelected=null}], text=null, _text=null}","{id=null, extension=null, coding=[{id=null, extension=null, system=http://terminology.hl7.org/CodeSystem/condition-ver-status, _system=null, version=null, _version=null, code=confirmed, _code=null, display=null, _display=null, userSelected=null, _userSelected=null}], text=null, _text=null}",,,"{id=null, extension=null, coding=[{id=null, extension=null, system=http://snomed.info/sct, _system=null, version=null, _version=null, code=840539006, _code=null, display=COVID-19, _display=null, userSelected=null, _userSelected=null}], text=COVID-19, _text=null}",,"{id=null, extension=null, reference=Patient/04c704c4-5d2d-4308-9c33-1690a6e47a6b, _reference=null, type=null, _type=null, identifier=null, display=null, _display=null}","{id=null, extension=null, reference=Encounter/2349e4b5-5163-425d-a444-b0e459a0260d, _reference=null, type=null, _type=null, identifier=null, display=null, _display=null}","2020-03-10T05:00:32-07:00",,,,,,,"2020-04-02T05:00:32-07:00",,,,,,,"2020-03-10T05:00:32-07:00",,,,,,
"""

print(f"len(sample_patient_resource): {len(sample_patient_resource)}")
print(f"len(sample_patient_encounters): {len(sample_patient_encounters)}")
print(f"len(sample_patient_observations): {len(sample_patient_observations)}")
print(f"len(sample_patient_conditions): {len(sample_patient_conditions)}")

## 3. Clinical Trial Patient Evaluation

```mermaid
flowchart LR
    A[Clinical Trial XML] -->|Parse| B[Trial Criteria JSON]
    C[Patient Resource] -->|Parse| D[Patient Profile]
    E[Patient Encounters] -->|Parse| D
    F[Patient Observations] -->|Parse| D
    G[Patient Conditions] -->|Parse| D
    B --> H{LLM Analysis}
    D --> H

    style A fill:#f9f,stroke:#333
    style B fill:#bbf,stroke:#333
    style H fill:#dfd,stroke:#333
```

### 3.1 Helper function for invoking an LLM using Amazon Bedrock

Here we define a helper function that leverages [Amazon Bedrock](https://aws.amazon.com/bedrock/) and provides an API to interact with Large Language Models (LLM). Here we have chosen to leverage ***Claude 2.1***. We also provide an estimate of the number of input tokens which measures the length of the input prompt to the LLM. We estimate the number of tokens by dividing the total character count by 6, which provides a rough approximation. ***Claude 2.1*** supports a 200,000 input token context limit which roughly translates to 500 pages, or about the size of a book.

In [ ]:
BEDROCK_CLIENT = boto3.client(service_name='bedrock-runtime')

def invoke_claude_v2(input_prompt):
    body = json.dumps({
        "prompt": f"\n\nHuman: {input_prompt} \n\nAssistant:",
        "max_tokens_to_sample": 1000,
        "temperature": 0,
    })

    model_id = 'anthropic.claude-v2:1'
    accept = 'application/json'
    contentType = 'application/json'

    concat_index = 1500
    if VERBOSE_LOGGING or len(body) <= concat_index:
        print(f"Invoking {model_id} with input: {body}")
    else:
        index_to_concat = min(concat_index, len(body))
        print(f"Invoking {model_id} with concatenated input (set VERBOSE_LOGGING to True for full input): {body[0:index_to_concat]}\n\n...\n\n{body[-1*index_to_concat:]}")

    input_token_estimate = int(len(input_prompt)/6)
    print(f"\n\n Estimated input tokens: {input_token_estimate}")
    
    print("\n\n Response below (Please wait for the response output before proceeding to the next cell as the request may take time to process):\n")

    response = BEDROCK_CLIENT.invoke_model(body=body, modelId=model_id, accept=accept, contentType=contentType)

    response_body = json.loads(response.get('body').read())

    llm_response = response_body.get('completion')

    print(llm_response)

    output_token_estimate = int(len(llm_response)/6)
    print(f"\n\n Estimated output tokens: {output_token_estimate}")

    total_cost = input_token_estimate * .000008 + output_token_estimate * .000024
    print(f"\n\n Estimated total cost: ${total_cost}")

    return llm_response

# Validate bedrock model access
try:
    invoke_claude_v2("ping, respond with \"pong\"")
    print("\n")
    print("Bedrock model access confirmed")
except Exception as e:
    print(e)
    print("\n")
    print("Bedrock model access failed. Please ensure you have the correct permissions and the model is available in your region. Follow the documentation at https://docs.aws.amazon.com/bedrock/latest/userguide/model-access-modify.html to add model access.")

### 3.2 Binary Clinical Trial Patient Evaluation

```mermaid
flowchart TB
    subgraph Binary
    A[Patient Data] -->|Simple Prompt| B{Good/Bad}
    B -->|Yes| C[Good Candidate]
    B -->|No| D[Bad Candidate]
    end
```

The first input prompt we use tries to solve our business challenge with a single prompt. 

We give Claude all of our patient data and follow foundational Claude prompt engineering best practices. 

1. We ensure we start our prompt with a background information on what the Assistant is trying to perform.
2. We tell the Assistant and describe what information is inside of the XML tags in our prompt.
3. We use a python formatted string literal (f-string) which inserts the python output of anything within ```{}``` in to the string. An example of this would be ```<tag>{sample_patient_resource}</tag>``` is replaced with ```<tag>{"identifier":[{"system":"https://github.com/synthetichealth/synthea","value":"04c704c4-5d2d-4308-9c33-1690a6e47a6b"} ... ,"resourceType":"Patient","meta":{"lastUpdated":"2023-11-17T01:57:03.687Z"}}</tag>```.
4. We then ask the Assistant the exact question we need the Assistant to answer
5. We ask the Assistant to provide the answer to the question inside of our user-defined XML tag for easy retreival later
6. We then provide the Assistant with example outputs

In [ ]:
input_prompt = f"""

We are looking to evaluate if a patient is a good candidate based on the criteria for a scientific study.

Inside <patient></patient> XML tags is the patient profile. 
Inside <encounter></encounter> XML tags is a list of the patient's encounters with healthcare providers.
Inside <observation></observation> XML tags is a list of the patient's observations from healthcare providers.
Inside <condition></condition> XML tags is a list of the patient's conditions diagnosed by healthcare providers.
Inside <study></study> XML tags is the criteria of a scientific study

<patient>{sample_patient_resource}</patient>

<encounter>{sample_patient_encounters}</encounter>

<observation>{sample_patient_observations}</observation>

<condition>{sample_patient_conditions}</condition>

<study>{criteria}</study>

Please provide your response for the patient as "good candidate" or "bad candidate". Put your answer to the user inside <answer></answer> XML tags

Here are 2 examples:
<example>
good candidate
</example>

<example>
bad candidate
</example>

"""

invoke_claude_v2(input_prompt);

#### 3.2.1 Binary Clinical Trial Patient Evaluation Analysis

We find that the Assistant is very conservative and any time the Assistant is unable to find postive evidence of every single study criteria, the result that is returned back is "bad candidate". If we were to use this input prompt for all of our patient repository, we would be unlikely to identify any candidate as a "good candidate". This result would not be valuable for our business so let's explore how to change our input prompt to better support our business objectives.

### 3.3 Categorical Clinical Trial Patient Evaluation

```mermaid
flowchart TB
    subgraph Categorical
    E[Patient Data] -->|Enhanced Prompt| F{Categories}
    F -->|A| G[Definite Match]
    F -->|B| H[Likely Match]
    F -->|C| I[Unlikely Match]
    F -->|D| J[Not a Match]
    end
```    

The second prompt changes the binary output of the LLM to be a categorical response. We ask the LLM to provide its answer as a multiple choice response which describe a range of candidate viability for the study.

In [ ]:
input_prompt = f"""

We are looking to evaluate if a patient is a good candidate based on the criteria for a scientific study.

Inside <patient></patient> XML tags is the patient profile. 
Inside <encounter></encounter> XML tags is a list of the patient's encounters with healthcare providers.
Inside <observation></observation> XML tags is a list of the patient's observations from healthcare providers.
Inside <condition></condition> XML tags is a list of the patient's conditions diagnosed by healthcare providers.
Inside <study></study> XML tags is the criteria of a scientific study

<patient>{sample_patient_resource}</patient>

<encounter>{sample_patient_encounters}</encounter>

<observation>{sample_patient_observations}</observation>

<condition>{sample_patient_conditions}</condition>

<study>{criteria}</study>

Please provide your response for the patient as

(A) Yes this patient is a good candidate for the scientific study.
(B) It is likely that this patient is a good candidate for the scientific study but more information might be needed.
(C) It is unlikely that this patient is a good candidate for the scientific study but more information might be needed.
(D) No this patient is not a good candidate for the scientific study.

Put your answer to the user inside <answer></answer> XML tags

Here are 4 examples:
<example>
A
</example>

<example>
B
</example>

<example>
C
</example>

<example>
D
</example>

"""

invoke_claude_v2(input_prompt);

#### 3.3.1 Categorical Clinical Trial Patient Evaluation Analysis

We now find that the Assistant is able to categorize patients into more precise buckets, and the same patient information that saw the result of a ***bad candidate*** now becomes ***B*** *It is likely that this patient is a good candidate for the scientific study but more information might be needed.* By allowing the LLM more options in the output, it is more likely to be precise and deliver helpful responses.

This brings us closer to meeting the business objective, and if we were to use this input prompt for our entire patient repository, we would end up with many patients distributed across these categorical buckets. 

What if our study is only recruiting for a small number of patients? We may still have hundreds of patients in the categories identified as good candidates. We need to further refine our results to provide quantitative data to establish a ranking of patients. We can achieve this goal by leveraging LLMs as part of a larger data analytics workflow to combine the strengths of traditional computing and Generative AI. We can use Generative AI to provide information about the study criteria, but add a supporting traditional data analytics workflow to use the analysis for a patient of each study criteria and translate that to a discrete patient suitability score.

## 4. Combining traditional data analytics and Generative AI

### 4.1 Parsing Clinical Trail Eligibility

The eligibility criteria that we have when downloading from the clinicaltrials.gov website comes as a text block. This can contain new line characters, bullet points, etc. We first want to transform this into a structured format like a JSON object, where we can use code to iterate through each study criteria. We could potentially write a rules-based engine to perform this work, but there are hundreds of variations in how researchers describe study criteria. Instead, we will leverage an LLM Assistant to perform this transformation.

In [ ]:
input_prompt = f"""

We are looking to process the inclusion and exclusion criteria for a scientific study as a json object. 
Inside <study></study> XML tags is the inclusion and exclusion criteria for a scientific study. 
Put your answer to the user inside <answer></answer> XML tags

<study>{criteria}</study>
"""

input_prompt += """

Here is an example:
<example>
{

"Inclusion Criteria":[
"Inclusion Criteria 1",
"Inclusion Criteria 2"],
"Exclusion Criteria":[
"Exclusion Criteria 1",
"Exclusion Criteria 2"],

}
</example>
"""

llm_study_response = invoke_claude_v2(input_prompt)

### 4.1.1 Parsing Clinical Trail Eligibility Response from LLM to Python object

We will now process the output of the LLM assistant and make it possible to interact with the structured output as a python json object.

In [ ]:
raw_claude_output = llm_study_response.replace("\n","")
xml_start_loc = raw_claude_output.find("<answer>") + 8
xml_end_loc = raw_claude_output.find("</answer>")

if xml_start_loc == -1 or xml_end_loc == -1:
    raise Exception("No <answer> xml tag detected in claude output")

study_criteria_json = json.loads(raw_claude_output.replace("\n","")[xml_start_loc:xml_end_loc])
print(json.dumps(study_criteria_json, indent=4))

### 4.2 Categorical Single Criteria Clinical Trial Patient Evaluation Analysis

```mermaid
flowchart TB
    subgraph Per-Criteria
    K[Patient Data] -->|Split Evaluation| L{Criteria 1}
    K -->|Split Evaluation| M{Criteria 2}
    K -->|Split Evaluation| N{Criteria n}
    
    L --> |A| O[Definite Match]
    L --> |B| P[Likely Match]
    L --> |C| Q[Unlikely Match]
    L --> |D| R[Not a Match]
    
    O & P & Q & R --> F1[Criteria 1 Final Result<br/>and Explanation]
    
    M --> |A| S[Definite Match]
    M --> |B| T[Likely Match]
    M --> |C| U[Unlikely Match]
    M --> |D| V[Not a Match]
    
    S & T & U & V --> F2[Criteria 2 Final Result<br/>and Explanation]
    
    N --> |...| W[...]
    W --> Fn[Criteria n Final Result<br/>and Explanation]
    end
```

We now ask the LLM assistant to provide a categorical response for each of the study criteria based on the patient profile. We also ask the LLM assistant to provide an explanation with references to the patient information for why that response was chosen. We want the answer to be outputted as a structured JSON object for easy analysis.

In [ ]:
input_prompt = f"""

We are looking to evaluate if a patient is a good candidate based on the criteria for a scientific study.

Inside <patient></patient> XML tags is the patient profile. 
Inside <encounter></encounter> XML tags is a list of the patient's encounters with healthcare providers.
Inside <observation></observation> XML tags is a list of the patient's observations from healthcare providers.
Inside <condition></condition> XML tags is a list of the patient's conditions diagnosed by healthcare providers.
Inside <study></study> XML tags is the criteria of a scientific study in JSON format

<patient>{sample_patient_resource}</patient>

<encounter>{sample_patient_encounters}</encounter>

<observation>{sample_patient_observations}</observation>

<condition>{sample_patient_conditions}</condition>

<study>{json.dumps(study_criteria_json, indent=4)}</study>

Please provide your response for each of the study criteria as

(A) Yes this patient is a good candidate for the scientific study.
(B) It is likely that this patient is a good candidate for the scientific study but more information might be needed.
(C) It is unlikely that this patient is a good candidate for the scientific study but more information might be needed.
(D) No this patient is not a good candidate for the scientific study.

and an explanation with references to the patient information for why you chose each response

in a JSON reponse.

If there is no evidence in the patient record of a specific condition, select either A or D.

Put your answer to the user inside <answer></answer> XML tags

"""
input_prompt += """
Here is an example:
<example>
{
    "Inclusion Criteria":{
        "Age between 12-55": {
            "Answer":"A",
            "Explanation":"The person's birthdate was 1996-12-10 which makes the patient 16 years old"
        },
        "History of Disease A":{
            "Answer":"B",
            "Explanation":"There is evidence that the person has had Disease A in the encounter on 2022-01-31"
        }
    },
    "Exclusion Criteria":{
        "Age greater than 55": {
            "Answer":"A",
            "Explanation":"The person's birthdate was 1996-12-10 which makes the patient 16 years old which is not an age greater than 55"
        },
        "History of Disease B": {
            "Answer":"C",
            "Explanation":"I did not find evidence of the patient for Disease B in the previous encounters but they had a related disease of Disease C as referenced by the doct's visit on 2021-10-13"
        },
    }
}
</example>

"""

llm_study_response = invoke_claude_v2(input_prompt)

### 4.3 Flatten Result Output

We will now parse the response of the LLM assistant which is currently a string of a multi-level JSON object. To make it easier to perform analysis, we will flatten this multi-level object to a single level by combining each of the levels of the JSON object and using a ```_``` character.

An example of this is transforming 

```
"Exclusion Criteria": {
        "Known adverse reaction or tolerance to study medication": {
            "Answer": "good candidate",
            "Explanation": "There is no information provided about any known adverse reactions or tolerances to study medications."
        },
        "Headache due to trauma": {
            "Answer": "good candidate",
            "Explanation": "There is no evidence in the patient record of a headache due to trauma."
        }
}
```

to

```
{
    Exclusion Criteria_Known adverse reaction or tolerance to study medication_Answer : "good candidate",
    Exclusion Criteria_Known adverse reaction or tolerance to study medication_Explanation : "There is no information provided about any known adverse reactions or tolerances to study medications.",
    Exclusion Criteria_Headache due to trauma_Answer : "good candidate",
    Exclusion Criteria_Headache due to trauma_Explanation : "There is no evidence in the patient record of a headache due to trauma."
}
```

In [ ]:
raw_claude_output = llm_study_response.replace("\n","")
xml_start_loc = raw_claude_output.find("<answer>") + 8
xml_end_loc = raw_claude_output.find("</answer>")

if xml_start_loc == -1 or xml_end_loc == -1:
    raise Exception("No <answer> xml tag detected in claude output")

def flat_keys(obj, new_obj={}, keys=[]):
    for key, value in obj.items():
        if isinstance(value, dict):
            # call the function again if the value is a dict
            # we go one step deeper: obj[key]
            # give the new_obj (by reference, so each call edit the *same* object)
            # give to used keys: keys + [key]
            flat_keys(obj[key], new_obj, keys + [key])
        else:
            new_obj['_'.join(keys + [key])] = value
    return new_obj

flattened_patient_result = flat_keys(json.JSONDecoder().decode(raw_claude_output[xml_start_loc:xml_end_loc]))
for key, value in flattened_patient_result.items():
    print(f"{key} : {value}")

### 4.4 Generate Patient Suitability Score

```mermaid
flowchart TD
    A[Patient Evaluation] --> B{Per Criteria Analysis}
    B --> C[Response A: +3 points]
    B --> D[Response B: +2 points]
    B --> E[Response C: +1 point]
    B --> F[Response D: 0 points]
    C & D & E & F --> G[Sum All Points]
    G --> H[Divide by Max Possible]
    H --> I[Multiply by 100]
    I --> J[Final Percentage]

    style A fill:#f9f,stroke:#333
    style B fill:#dfd,stroke:#333
    style G fill:#bbf,stroke:#333
    style J fill:#f96,stroke:#333
```

We can now use the information we have about each study criteria for a patient to generate a discrete patient suitability score. We calculate this by adding up the score that the patient receives for each study criteria using the formula below

| LLM Response of Study Criteria    | Score |
| -------- | ------- |
| (A) Yes this patient is a good candidate for the scientific study. | 3 |
| (B) It is likely that this patient is a good candidate for the scientific study but more information might be needed. | 2 |
| (C) It is unlikely that this patient is a good candidate for the scientific study but more information might be needed. | 1 |
| (D) No this patient is not a good candidate for the scientific study. | 0 |

In [ ]:
total_possible_score = 0
patient_score = 0
for criteria, result in flattened_patient_result.items():

    if criteria.endswith("_Answer"):
        total_possible_score += 3
    
        match result:
            case "A":
                patient_score += 3
            case "B":
                patient_score += 2
            case "C":
                patient_score += 1
            case "D":
                patient_score += 0
            case _:
                raise Exception(f"Unexpected result {result} in flattened_patient_result")

patient_suitability_percentage = round((patient_score/total_possible_score) * 100, 2)

print(f"patient_score: {patient_score}, total_possible_score: {total_possible_score}, patient_suitability_percentage: {patient_suitability_percentage}%")

#### 4.4.1 Patient Suitability Score Explanation

Using this traditional data analytics technique, we can now calculate a discrete patient suitability percentage, which can be used to rank patients in an ordered list of those most likely to be suitable for a clinical study to help your business prioritize which candidates to review and help complete your cohort identification analysis.